In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib as mpl 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import cluster, preprocessing, linear_model, tree, model_selection, feature_selection
from sklearn import base, ensemble, decomposition, metrics, pipeline, datasets, impute
from skopt import gp_minimize, space, gbrt_minimize, dummy_minimize, forest_minimize
from functools import partial
import os
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn import ensemble, preprocessing, tree, model_selection, feature_selection, pipeline, metrics, svm
from imblearn import under_sampling, over_sampling, combine
from imblearn import pipeline as imb_pipeline
from imblearn import ensemble as imb_ensemble
from sklearn.model_selection import StratifiedKFold

!pip install rfpimp


/kaggle/input/janatahack-customer-segmentation/Test.csv
/kaggle/input/janatahack-customer-segmentation/sample_submission.csv
/kaggle/input/janatahack-customer-segmentation/Train.csv
     |████████████████████████████████| 160 kB 5.3 MB/s 
  Created wheel for rfpimp: filename=rfpimp-1.3.5-py3-none-any.whl size=10257 sha256=0390d74993f42c77f39a611e04c0363567194def691c6de56fce34cb786452c3
  Stored in directory: /root/.cache/pip/wheels/15/88/ae/4e8850abc5088fd7e50cc7f869450d6085a0ec6d245adf8874
  Created wheel for stratx: filename=stratx-0.4.1-py3-none-any.whl size=35079 sha256=cac1b69c90bc8caa4f6614daa22d922d0b78f81ad4b4445e33158bb166712f93
  Stored in directory: /root/.cache/pip/wheels/88/85/01/fa1a3072052c9b027e3c40654b5dc8a488be8afc07442f39e0
Successfully built rfpimp stratx
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
train = pd.read_csv('/kaggle/input/janatahack-customer-segmentation/Train.csv')
test = pd.read_csv('/kaggle/input/janatahack-customer-segmentation/Test.csv')

In [3]:
# View target balance/Imbalance
train['Segmentation'].value_counts()

D    2268
A    1972
C    1970
B    1858
Name: Segmentation, dtype: int64

In [4]:
mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
rev_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}

In [5]:
train.head(2)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A


In [6]:
## Label Encode and preprocess
train_copy  = train.copy()
test_copy = test.copy()
train_copy['tr'] = 1
test_copy['tr'] = 0

appended = pd.concat([train_copy, test_copy], axis = 0)

cat_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
label_enc = {}
for col in cat_cols:
    appended[col] = appended[col].astype(str)
    enc = preprocessing.LabelEncoder().fit(appended[col])
    appended[col] = enc.transform(appended[col])
    label_enc[col] = enc

cats = ['Gender', 'Ever_Married','Graduated','Profession','Spending_Score',
'Var_1']
appended = pd.get_dummies(appended, columns = cats)
##################### create features from ID column ##############
def id_features(data):
    df = data.copy()
    df['week'] = df['ID']%7
    df['month'] = df['ID']%30
    df['year'] = df['ID']%365
    df['num_weeks'] = df['ID']//7
    df['num_year'] = df['ID']//365
    df['num_quarter'] = df['ID']//90
    df['quarter'] = df['ID']%90
    df['num_days'] = df['ID'].values - 458982
    df['num_weeks_2'] = (df['ID'].values - 458982)//7
    df['num_months_2'] = (df['ID'].values - 458982)//30

    return df
def id_features(data):
    df = data.copy()
    df['week'] = df['ID']%7
    df['month'] = df['ID']%30
    df['year'] = df['ID']%365
    df['quarter'] = df['ID']%90


    return df
appended = id_features(appended)
#appended = pd.get_dummies(appended, columns = cat_cols)
train_copy = appended.loc[appended['tr'] == 1]
test_copy = appended.loc[appended['tr'] == 0]
Xcols = appended.drop(['Segmentation', 'tr'], axis = 1).columns
'''Xcols = ['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1']'''
ycol = 'Segmentation'

X = train_copy[Xcols]
y = train_copy[ycol]

Xtest = test_copy[Xcols]

# Tune Hyperparameters - skopt

In [7]:
############ Tune Random Forest
def optimize_sk(params, param_names, X, y, scoring, estimator, cv = model_selection.StratifiedKFold(n_splits = 5)):
    '''params: list of param values
    param_names: param names
    x: training exogs
    y: training endogs
    return: negative metric after k fold validation'''

    params = dict(zip(param_names, params))

    # Initialize the model
    model = estimator(**params)

    kf = cv

    scores = []
    for train_index, test_index in kf.split(X, y):
        # Split Data
        X_train, y_train = np.array(X)[train_index, :], y[train_index]
        X_test, y_test = np.array(X)[test_index, :], y[test_index]

        # Fit model
        im = impute.KNNImputer().fit(X_train)
        X_train = im.transform(X_train)
        model.fit(X_train, y_train)

        # Evaluate model
        preds = model.predict(im.transform(X_test))
        scores.append(scoring(y_test, preds))

    return -np.mean(scores)

# Scoring
def f1_score(y_true, y_pred):
    return metrics.f1_score(y_true, y_pred, average = 'macro')

def accuracy(y_true, y_pred):
    return metrics.accuracy_score(y_true, y_pred)

# Parameter Space
param_space = [
    space.Integer(100, 1000, name = 'n_estimators'),
    space.Integer(2, 25, name = 'max_depth'),
    space.Real(0, 1, name = 'max_features'),
    space.Integer(2, 25, name = 'min_samples_leaf'),
    space.Categorical(['gini', 'entropy'], name = 'criterion'),
    space.Categorical([None, 'balanced', 'balanced_subsample'], name = 'class_weight'),
    space.Categorical([True, False], name = 'bootstrap')
]

# Param names
names = ['n_estimators', 'max_depth', 'max_features', 'min_samples_leaf', 'criterion', 'class_weight', 'bootstrap']

cat_cols =  ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1', 'ID']
cat_cols =  ['Var_1']

# Define objective - reformat it in terms of what is required for skopt
objective_optimization = partial(optimize_sk, param_names = names, X = X, y = y, 
                                scoring = accuracy, estimator = partial(ensemble.RandomForestClassifier, n_jobs = -1, random_state = 0))

# Perform Optimization
#gbrt_minimize, dummy_minimize, forest_minimize
'''skopt_optimization = gp_minimize(func = objective_optimization, 
                                dimensions = param_space, n_calls = 10, n_random_starts = 10, 
                                x0 = None, y0 = None, random_state = 10, 
                                verbose = 10)'''
skopt_optimization = dummy_minimize(func = objective_optimization, 
                                dimensions = param_space, n_calls = 10,
                                x0 = None, y0 = None, random_state = 10, 
                                verbose = 10)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 13.9384
Function value obtained: -0.5265
Current minimum: -0.5265
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 41.2777
Function value obtained: -0.5346
Current minimum: -0.5346
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 22.2430
Function value obtained: -0.5275
Current minimum: -0.5346
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 57.4814
Function value obtained: -0.5306
Current minimum: -0.5346
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 16.6387
Function value obtained: -0.5316
Current minimum: -0.5346
Iteration No: 6 star

In [8]:
model = pipeline.make_pipeline(impute.KNNImputer(), 
                               ensemble.RandomForestClassifier(**dict(zip(names, skopt_optimization.x)), 
                                                               n_jobs = -1, random_state = 0)).fit(X, y)

In [9]:
dict(zip(names, skopt_optimization.x))

{'n_estimators': 981,
 'max_depth': 10,
 'max_features': 0.26556612677209707,
 'min_samples_leaf': 12,
 'criterion': 'gini',
 'class_weight': 'balanced_subsample',
 'bootstrap': False}

# Ensemble and perform hand Tuning. Tuning models by hand gave me better results.

In [10]:
from sklearn import impute, pipeline

In [11]:
model_cb = cb.CatBoostClassifier( verbose = False)

model_lgb = lgb.LGBMClassifier(n_estimators = 1000, min_samples_in_leaf = 10, learning_rate = .02, 
                          feature_fraction = .8, max_depth = 8)

# Soft Voting Classifier
model_voting = ensemble.VotingClassifier([('catboost', model_cb), ('lightgbm', model_lgb)], 
                                         voting = 'soft').fit(X, y)

# The best pipelines so far:
### * kNN Imputation + RandomForest
### * Lightgbm

In [12]:
# .94
model4 = pipeline.make_pipeline(impute.KNNImputer(n_neighbors = 10), ensemble.RandomForestClassifier(class_weight = 'balanced_subsample',
                    n_estimators = 200, max_depth = 20, criterion = 'entropy', max_features = .8, oob_score = True, random_state = 0)).fit(X, y)

In [13]:
# .94
model2 = lgb.LGBMClassifier(n_estimators=300, max_features = .85, max_depth = 15, learning_rate = 1.1).fit(X, y)

In [14]:
###### View Feature importance  -  Using Permutation Importance
import rfpimp

imp = rfpimp.importances(model2, X, y)
imp

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


,Importance
Feature,
Age,0.1734
ID,0.1438
year,0.1062
quarter,0.0622
month,0.0328
Work_Experience,0.0290
Family_Size,0.0270
Profession_0,0.0150
Profession_5,0.0132


In [15]:
pred = pd.DataFrame()
pred['ID'] = test['ID'].values
pred['Segmentation'] = pd.Series(model2.predict(Xtest))
pred.to_csv('Seg.csv', index = None)